In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
f = pd.read_csv("lfw20.csv", header = None)
f.head()

,0,1,2,3,4,5,6,7,8,9,...,5646,5647,5648,5649,5650,5651,5652,5653,5654,5655
0,0,45.0,46.000,44.0000,43.000,44.333,46.667,50.667,57.000,62.000,...,165.67,179.670,181.330,178.670,173.330,171.330,158.670,113.000,70.000,57.333
1,0,10.0,9.000,5.6667,5.000,7.000,10.000,16.000,18.000,20.667,...,121.33,113.000,94.000,63.000,31.000,20.667,21.333,20.667,20.333,23.000
2,0,15.0,18.667,16.6670,20.000,27.333,34.000,39.000,50.667,69.667,...,170.67,200.000,224.330,228.670,231.000,228.000,230.000,227.000,224.330,225.670
3,0,33.0,31.667,32.0000,31.333,32.333,38.333,41.333,42.000,52.667,...,50.00,50.333,50.333,50.333,50.667,50.667,50.667,50.333,50.000,50.000
4,0,33.0,33.333,29.6670,26.667,28.000,33.667,43.000,54.333,63.667,...,147.33,167.000,194.670,214.670,223.330,228.330,226.000,220.670,193.670,126.330


In [2]:
random.seed(0)
data = f.to_numpy()[:, 1:] / 255
Y = f.to_numpy()[:, 0]
#num = round(data.shape[0] * 2/3)#2/3rd data for training
trainX, testX = train_test_split(data, train_size = 0.66)#data[:num, :], data[num:,:]
trainY, testY = Y[:len(trainX)], Y[len(trainX):]
print(trainX.shape, testX.shape, trainY.shape, testY.shape)

(818, 5655) (422, 5655) (818,) (422,)


In [3]:
#Zscoring
ztrx = (trainX - np.mean(trainX, axis = 0)) / (np.std(trainX, axis = 0, ddof = 1))
ztsx = (testX - np.mean(trainX, axis = 0)) / (np.std(trainX, axis = 0, ddof = 1))
#Because we were asked to zscore test based on training data only
ztrx.shape, ztsx.shape, trainY.shape, testY.shape

((818, 5655), (422, 5655), (818,), (422,))

In [4]:
def KNN(a, b):
    return trainY[np.argmax(np.sum((np.tile(a, (b.shape[0], 1)) - b)**2, axis = 1))]

In [5]:
vals = []
for i in range(len(testY)):
    vals.append(KNN(ztsx[i], ztrx))
len(vals)

422

In [6]:
def get_accuracy(true, pred):
    count = 0
    if len(true) != len(pred):
        print("Error")
        return
    for i in range(len(true)):
        if true[i] == pred[i]:
            count += 1
    return round(count / len(true))

In [7]:
acc1 = get_accuracy(vals, testY)
print(f'Accuracy for k = 1 and D = original is {round(acc1 * 100, 2)}%')

Accuracy for k = 1 and D = original is 18.0%


In [8]:
def PCA(d):
    '''PCA for D = 100'''
    w, v  =np.linalg.eigh(np.cov(np.transpose(d)))
    return v[:, w.argsort()[-100:].tolist()[::-1]]

In [9]:
extra = PCA(trainX)

In [10]:
newtrainX_nw = trainX @ extra
newtestX_nw = testX @ extra

In [11]:
new_vals = []
for i in range(len(testY)):
    new_vals.append(KNN(newtestX_nw[i], newtrainX_nw))
len(new_vals)

422

In [12]:
acc2 = get_accuracy(testY, new_vals)
print(f'Accuracy for k = 1 and D = 100, with PCA(non-whitened) is {round(acc2 * 100, 2)}%')

Accuracy for k = 1 and D = 100, with PCA(non-whitened) is 21.0%


In [13]:
newtrainX_w = trainX @ (extra / np.sqrt(abs(extra)))
newtestX_w = testX @ (extra / np.sqrt(abs(extra)))

In [14]:
new_new_vals = []
for i in range(len(testY)):
    new_new_vals.append(KNN(newtestX_w[i], newtrainX_w))
len(new_new_vals)

422

In [16]:
acc3 = get_accuracy(testY, new_new_vals)
print(f'Accuracy for k = 1 and D = 100, with PCA(whitened) is {round(acc3 * 100, 2)}%')

Accuracy for k = 1 and D = 100, with PCA(whitened) is 26.0%


I don't know why but most(>90%) of the times I get all accuracies = 0, I am including an instance where all of them were not zero in my report